# Four model test with twitter_glove200

Data set:
1.	Text data from the audio calls.

2. Dataset from Workshop on Computational Approaches to Subjectivity, Sentiment and Social Media Analysis (WASSA-2017), which is to be held in conjunction with EMNLP-2017, which can be downloaded [here](http://saifmohammad.com/WebPages/EmotionIntensity-SharedTask.html)

Preprinted Glove:
-	Common Crawl 840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download): glove.840B.300d.zip, which can be downloaded [here](https://nlp.stanford.edu/projects/glove/). 

I have tested the performance of the following models:

1. Model1: LSTM (0.6120)
2. Model2: Bidirectional GRU(0.6279)
3. Model3: Single Convolutional Neural Network (0.5176)
4. Model4: Multi-size CNN (0.6005)

In [1]:
import sys, os, re, csv, codecs 
import numpy as np
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from glob import glob
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Merge
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
from keras.models import model_from_json
import pickle
from sklearn.utils import resample
import nltk
from nltk.stem import PorterStemmer
from multiprocessing import Pool

stemming = PorterStemmer()
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')


def stowords(words):
    words = str(words)
    words = words.lower()
    words = words.replace("what's", "what is ")
    words = words.replace("shouldn't", "should not ")
    words= words.replace("\'ve", " have ")
    words= words.replace("'s", " ")
    words= words.replace("can't", "cannot ")
    words= words.replace("don't", "do not ")
    words= words.replace("doesn't", "does not ")
    words= words.replace("n't", "not ")
    words = words.replace(r"i'm", "i am")
    words =words.replace(r" m ", " am ")
    words =words.replace(r"\'re", " are ")
    words =words.replace(r"\'d", " would ")
    words =words.replace(r"'ll", " will ") 
    
    words = words.replace("'", "")
    words= words.replace("!", "") 
    words =words.replace(r"´", "")
    words = words.replace("`", "")
    words = words.replace("\r\n", " ")
#     words = " ".join([stemmer.stem(word) for word in words.split()])
    
#     words = " ".join([stemming.stem(word) for word in words.split()])
#     words = " ".join([Word(word).lemmatize()for word in words.split()])
    
   
    letters_only = re.sub("[^a-zA-Zé]", " ", words)
    word = letters_only.lower().split()
 
    se = set(STOP_WORDS)
    meaningful_words =" ".join( [w for w in word if w not in se]) 
#     meaningful_words = str(TextBlob(meaningful_words).correct())
    
    return ( meaningful_words )

Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
df = pd.read_hdf('./Data/dataset2_tags_texts.hdf')

In [4]:
df.shape

(5670, 217)

In [5]:
df.head()

,gradable_id,polarity,subjectivity,text,Unnamed: 0,mfcc_1_m1_bin1,mfcc_2_m1_bin1,mfcc_3_m1_bin1,mfcc_4_m1_bin1,mfcc_1_m2_bin1,...,mfcc_feat_var_1,mfcc_feat_var_2,mfcc_feat_var_3,mfcc_feat_var_4,qa_score,tags,time,clean_tag,clean_text,text_lenght
0,418139,0.318571,0.500516,['(0.9353249073028564): hi thank you for calli...,922,8.075193,21.911402,17.653523,10.752469,18.578279,...,25.014487,193.926085,131.714832,212.163095,38,"[""shipping"", ""tracking_inquiry__processing""]",2017-03-13T15:43:02,shipping tracking inquiry processing,hi thank you for calling fabfitfun meetings le...,1345
1,1433069,0.262808,0.417980,['(0.7501416802406311): thanks again he resear...,4476,6.030269,22.738942,17.901995,12.519905,18.912234,...,21.230926,163.387397,124.742727,210.699407,93,"[""order__shipped__order_status""]",2017-05-08T14:51:16,order shipped order status,thanks again he researches chelsea how may hel...,1875
2,18834,0.234722,0.375000,['(0.9273091554641724): thank you for calling ...,62,6.775557,22.189641,17.218559,12.555730,18.399315,...,26.413125,267.558405,96.075137,191.671796,100,"[""application"", ""application_status"", ""non_sta...",2017-05-11T14:21:27,application application status non state employee,thank you for calling the state of tennessee d...,828
3,1498618,0.345833,0.462500,"[""(0.9570504426956177): thank you for calling ...",4850,5.875857,23.094995,16.586156,14.985884,17.504581,...,12.566649,142.067990,101.407129,145.302891,84,"[""shave_plans__cancel_plan__overstocked""]",2017-06-20T20:34:38,shave plans cancel plan overstocked,thank you for calling harry s my name is kayle...,736
4,1446730,0.087349,0.440519,"[""(0.9487425088882446): hi yes this is Zack fr...",4555,5.762013,22.092987,16.735214,20.478581,18.337259,...,18.309896,181.549078,105.479915,184.641216,75,"[""callback"", ""site__profile__edit_email""]",2017-05-17T19:08:16,callback site profile edit email,hi yes this is zack from harry s giving you a ...,1886


In [6]:
def cat(x):
    if x == 0:
        return ('bad')
    else:
        return ('good')

In [7]:
df['cat'] = df.category.apply(cat)

In [8]:
df = df.join(pd.get_dummies(df.cat))

In [9]:
df.columns

Index(['gradable_id', 'polarity', 'subjectivity', 'text', 'Unnamed: 0',
       'mfcc_1_m1_bin1', 'mfcc_2_m1_bin1', 'mfcc_3_m1_bin1', 'mfcc_4_m1_bin1',
       'mfcc_1_m2_bin1',
       ...
       'mfcc_feat_var_4', 'qa_score', 'tags', 'time', 'clean_tag',
       'clean_text', 'text_lenght', 'cat', 'bad', 'good'],
      dtype='object', length=220)

In [10]:
pool = Pool(processes = 28) # or some number of your choice
df['cleaner_text'] = pool.map(stowords, df.clean_text)
pool.terminate()

In [11]:
EMBEDDING_FILE = '/home/sohrab/streetbees/emotion_detection/crawl-300d-2M.vec'

## Reading GloVe vector:

In [12]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [13]:
# This model has vectors for 1193514 words 
len(embeddings_index)

570616

In [14]:
len(embeddings_index['what'])

300

In [15]:
embed_size = 300 # how big is each word vector
max_features = 45000 # how many unique words to use (this can be a hyperparameter)
maxlen = 50 # max number of words in a comment to use (this can be a hyperparameter)

In [16]:
train = df[['cleaner_text','bad','good']].copy().reindex()

## Text Tokenization: 

In [17]:
# train
list_sentences_train = train["cleaner_text"].fillna("_na_").values
y = train[['bad','good']].values

In [18]:
tokenizer = Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(list(list_sentences_train))

list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)

X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [19]:
train.cleaner_text[1]

'thanks researches chelsea help n hi chelsea richard griffith calling check sure order went double check price shipment sorry returning customer harry s quit come tried place order blaze guess butter shaving cream wanted sure went information correct check s n auburn tumbler gmail com n okay great let s n francia found account looks like email s gmx instead gmail email think s verify shipping address sure right account street montgomery alabama n yep found account think good order shipped sea n okay order process yesterday looks like haven t updated tracking information sent n anti leaves sent th n free trials heading way n okay set blades shipped th th right trials heading way n okay th email telling today shipped wait invoice order goes email days shipment email day ship reminder ll send email problem look forward great day thanks calling n'

In [20]:
X_t[1]

array([   1, 9297, 2093,   70,   39,    1,   66, 2188, 1701,   51,    1,
          3,   61,   60,  120,   39,   39,   18, 2188, 1701,   51,    1,
          3,   39,    9,  622,   19,  120,  245,  624,   22,  355,    9,
         54,   91,    9,   27,  124,  597,   13,   41,    9,   46,   47,
        277,   43,   27,   63,   17,    1], dtype=int32)

In [21]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((max_features, embed_size))

missing_words = []
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is None:
        missing_words.append(word)
    elif embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [22]:
len(embedding_vector)

300

In [23]:
#total number of words in our dataset:
len(word_index)

23607

## Model 1:

In [24]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(2, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])

In [25]:
model.fit(X_t, y, batch_size=128, epochs=2, validation_split=0.1);

Train on 5103 samples, validate on 567 samples
Epoch 1/2
5103/5103 [==============================] - 10s 2ms/step - loss: 0.6773 - acc: 0.5690 - val_loss: 0.6550 - val_acc: 0.5926
Epoch 2/2
5103/5103 [==============================] - 9s 2ms/step - loss: 0.6255 - acc: 0.6356 - val_loss: 0.6389 - val_acc: 0.6120


## Model 2:

In [26]:
inp = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(70, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
outp = Dense(2, activation="sigmoid")(conc)
lr = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.001)   
model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy',
                  optimizer=lr,
                  metrics=['accuracy'])

In [27]:
model.fit(X_t, y, 
          batch_size=16, 
          epochs=3, 
          validation_split=0.1);

Train on 5103 samples, validate on 567 samples
Epoch 1/3
5103/5103 [==============================] - 18s 3ms/step - loss: 0.6689 - acc: 0.5827 - val_loss: 0.6475 - val_acc: 0.5917
Epoch 2/3
5103/5103 [==============================] - 15s 3ms/step - loss: 0.6135 - acc: 0.6515 - val_loss: 0.6268 - val_acc: 0.6296
Epoch 3/3
5103/5103 [==============================] - 16s 3ms/step - loss: 0.5493 - acc: 0.7177 - val_loss: 0.6184 - val_acc: 0.6279


# Model 3:

### Single convolutional neural network

In [24]:
inp = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Dropout(0.6)(x)
x = Conv1D(64, 5, padding='same', activation='relu')(x)
x = Dropout(0.2)(x)
x = MaxPooling1D()(x)
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(2, activation='sigmoid')(x)

model = Model(inputs=inp, outputs=x)
optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(loss='binary_crossentropy', 
              optimizer=optim, 
              metrics=['accuracy'])

In [25]:
model.fit(X_t, y, batch_size=16, epochs=3, validation_split=0.1);

Train on 5103 samples, validate on 567 samples
Epoch 1/3
5103/5103 [==============================] - 2s 438us/step - loss: 0.7060 - acc: 0.5012 - val_loss: 0.6930 - val_acc: 0.5159
Epoch 2/3
5103/5103 [==============================] - 1s 218us/step - loss: 0.6924 - acc: 0.5129 - val_loss: 0.6924 - val_acc: 0.5088
Epoch 3/3
5103/5103 [==============================] - 1s 221us/step - loss: 0.6872 - acc: 0.5414 - val_loss: 0.6871 - val_acc: 0.5176


# Model 4:

### Multi-size CNN:

This is an implementation of a multi-size CNN as shown in Ben Bowle's excellent blog [post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [26]:
graph_in = Input ((maxlen, 300))
convs = [ ] 
for fsz in range (3, 6): 
    x = Conv1D(64, fsz, padding='same', activation='relu')(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode="concat")(convs) 
graph = Model(graph_in, out) 

/home/sohrab/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [27]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = graph(x)
x = Dropout(0.2)(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(2, activation='sigmoid')(x)

model = Model(inputs=inp, outputs=x)
optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(loss='binary_crossentropy', 
              optimizer=optim, 
              metrics=['accuracy'],
             )

In [29]:
model.fit(X_t, y, 
          batch_size=16, 
          epochs=2, 
          validation_split=0.1,
         );

Train on 5103 samples, validate on 567 samples
Epoch 1/2
5103/5103 [==============================] - 5s 978us/step - loss: 0.5510 - acc: 0.7173 - val_loss: 0.6330 - val_acc: 0.6058
Epoch 2/2
5103/5103 [==============================] - 5s 977us/step - loss: 0.3096 - acc: 0.8790 - val_loss: 0.7871 - val_acc: 0.6005
